In [7]:
%matplotlib qt

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Inputs ---
excel_path = "N87_Permeability.xlsx"      # change if needed
output_path = "N87_Permeability_interp_FIX2.txt"

# --- Load ---
df = pd.read_excel(excel_path)

# --- Clean Series1 (Mu') source: remove nonpositive freq / NaNs, sort, dedupe ---
s1 = df[["Series1.X", "Series1.Y"]].copy()
s1 = s1[(s1["Series1.X"] > 0) & (s1["Series1.Y"].notna())]
s1 = s1.sort_values("Series1.X").drop_duplicates(subset="Series1.X", keep="last")

# Convert frequency from kHz -> Hz
x1_hz = s1["Series1.X"].astype(float).to_numpy() * 1e3  # original μ′ freqs (Hz)
y1    = s1["Series1.Y"].astype(float).to_numpy()        # original μ′

# Reference frequency axis & μ″ from Series2, convert kHz -> Hz
f_hz = pd.to_numeric(df["Series2.X"], errors="coerce").to_numpy() * 1e3  # Hz
mu2  = pd.to_numeric(df["Series2.Y"], errors="coerce").to_numpy()        # μ″

# Keep only finite, positive frequencies on the reference axis
m_ref = np.isfinite(f_hz) & (f_hz > 0) & np.isfinite(mu2)
f_hz = f_hz[m_ref]
mu2  = mu2[m_ref]

# --- Interpolate μ′ onto the reference freq axis (Hz) ---
# Inside range: interpolate; below min: hold first value; above max: set to 0
mu1 = np.interp(f_hz, x1_hz, y1, left=y1[0], right=y1[-1])
mu1[f_hz > x1_hz.max()] = 0.0

# --- Save (no headers, tab-separated). 'f' is in SI units (Hz) ---
out = pd.DataFrame({"f": f_hz, "Mu'": mu1, "Mu''": mu2})
out.to_csv(output_path, sep="\t", index=False, header=False, float_format="%.9f")
print(f"Saved: {output_path} (frequency in Hz)")

# --- Plot complex permeability ---
plt.figure(figsize=(10, 5))

# μ′: original dots (Series1) and interpolated line (on Series2 grid)
plt.semilogx(x1_hz, y1, 'o', label="μ′ original (Series1)", markersize=4, alpha=0.7)
plt.semilogx(f_hz,  mu1, '-', label="μ′ interpolated → Series2 grid", linewidth=2)

# μ″: original dots (Series2) and connected line
plt.semilogx(f_hz, mu2, 'o', label="μ″ original (Series2)", markersize=4, alpha=0.7)
plt.semilogx(f_hz, mu2, '-', label="μ″ line", linewidth=1)

plt.xlabel("Frequency [Hz]")
plt.ylabel("Permeability")
plt.title("Complex Permeability vs Frequency (original dots & interpolated lines)")
plt.grid(True, which="both", ls="-", alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


Saved: N87_Permeability_interp_FIX2.txt (frequency in Hz)


In [9]:
# -*- coding: utf-8 -*-
"""
Initial B(H) analytical curves from hysteresis — sweep 'c' values (CST-like)

Input (Excel with X=H [A/m], Y=B [mT]):  BH_FILE = "N87_BH.xlsx"
Model: B(H) = μ0 H + J(H),  J(H) = Js * x / (1 + x^c)^(1/c),  x = H / H0,  H0 = Js/(μ0(μi-1))

What it does:
  • Extracts a clean first-quadrant, monotone initial curve (used only for parameter estimation)
  • Estimates μᵢ (low-field slope) and Jₛ (high-field asymptote) from that initial curve
  • Sweeps c ∈ [C_MIN, C_MAX] with N_C values, plots curves, exports one CST file per c
  • Raw B–H scatter is large and drawn last (on top). The extracted curve is NOT plotted.
"""

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from pathlib import Path

# ---------------- User controls ----------------
BH_FILE    = "N87_BH.xlsx"

# c sweep
C_MIN      = 2
C_MAX      = 3
N_C        = 5

# Estimation knobs (on extracted initial curve)
LOW_FIELD_FRAC  = 0.02   # fraction of max(H_init) for μi slope fit
HIGH_FIELD_FRAC = 0.90   # lower bound fraction of H_init range for Js estimate

# Export
EXPORT_ALL = True
OUT_DIR    = Path("out")
N_EXPORT   = 500
H_MAX_FACTOR = 1.05

# Plot style
FIG_SIZE    = (10, 7)
FONT_SIZE   = 18
LEGEND_SIZE = 14
LINE_WIDTH  = 2.4
MARKER_SIZE = 10  # bigger scatter

# ------------------------------------------------
mu0 = 4e-7 * math.pi

def load_bh_excel(path: str | Path):
    df = pd.read_excel(path)
    H = pd.to_numeric(df["X"], errors="coerce").to_numpy()
    BmT = pd.to_numeric(df["Y"], errors="coerce").to_numpy()
    m = np.isfinite(H) & np.isfinite(BmT)
    H = H[m]; B = (BmT[m] * 1e-3)
    idx = np.argsort(H)
    return H[idx], B[idx]

def extract_initial_from_bh(H: np.ndarray, B: np.ndarray):
    """First quadrant ascending branch; enforce monotone B(H)."""
    m = H >= 0
    Hq = H[m]; Bq = B[m]
    keepH, keepB = [], []
    lastH, lastB = -1e99, -1e99
    for h, b in zip(Hq, Bq):
        if h > lastH:
            if b < lastB:
                b = lastB
            keepH.append(float(h)); keepB.append(float(b))
            lastH, lastB = h, b
    Hs = np.array(keepH, float)
    Bs = np.array(keepB, float)
    return Hs, Bs

def linear_fit(x, y):
    A = np.vstack([x, np.ones_like(x)]).T
    m, b = np.linalg.lstsq(A, y, rcond=None)[0]
    return float(m), float(b)

def estimate_mu_i(Hs: np.ndarray, Bs: np.ndarray):
    if Hs.size < 4:
        return 1000.0
    Hcut = max(1e-9, LOW_FIELD_FRAC * Hs.max())
    sel = Hs <= Hcut
    if sel.sum() < 6:
        k = min(max(6, int(0.1*Hs.size)), Hs.size)
        x, y = Hs[:k], Bs[:k]
    else:
        x, y = Hs[sel], Bs[sel]
    s0, _ = linear_fit(x, y)
    return max(1.0, s0 / mu0)

def estimate_Js(Hs: np.ndarray, Bs: np.ndarray):
    if Hs.size < 5:
        return float(np.max(Bs) - mu0*np.max(Hs))
    Hthr = Hs.min() + HIGH_FIELD_FRAC * (Hs.max() - Hs.min())
    sel = Hs >= Hthr
    if sel.sum() < 5:
        sel = np.zeros_like(Hs, dtype=bool)
        sel[-max(5, Hs.size//10):] = True
    J = Bs[sel] - mu0*Hs[sel]
    return float(np.mean(np.clip(J, 0.0, None)))

def analytical_J(H: np.ndarray, mu_i: float, Js: float, c: float):
    H0 = Js / (mu0 * max(mu_i - 1.0, 1e-9))
    x = np.maximum(H, 0.0) / max(H0, 1e-30)
    return Js * (x / np.power(1.0 + np.power(x, c), 1.0/c))

# ---------------- Pipeline ----------------
# Load raw BH and build initial curve (used for parameter estimation only)
H_raw, B_raw = load_bh_excel(BH_FILE)
H_init, B_init = extract_initial_from_bh(H_raw, B_raw)

# Estimate μi and Js from the extracted initial curve
mu_i = estimate_mu_i(H_init, B_init)
Js   = estimate_Js(H_init, B_init)

# Prepare H grid for model curves and (optional) exports
Hmax = H_init.max() * H_MAX_FACTOR if H_init.size else H_raw.max() * H_MAX_FACTOR
H_grid = np.linspace(0.0, max(Hmax, 1.0), N_EXPORT)

# c values
cs = np.linspace(C_MIN, C_MAX, N_C)

# Export directory
if EXPORT_ALL:
    OUT_DIR.mkdir(parents=True, exist_ok=True)

# Plot
plt.figure(figsize=FIG_SIZE)

for c in cs:
    B_model = mu0*H_grid + analytical_J(H_grid, mu_i, Js, float(c))
    # export per c
    if EXPORT_ALL:
        fname = OUT_DIR / f"initial_BH_c_{c:.3f}.txt"
        pd.DataFrame({"H": H_grid, "B": B_model}).to_csv(
            fname, sep="\t", index=False, header=False, float_format="%.9f"
        )
    # plot curves
    plt.plot(H_grid, B_model, lw=LINE_WIDTH, label=f"Analytical (c={c:.2f})")

# draw scatter LAST so it's on top
plt.scatter(H_raw, B_raw, s=MARKER_SIZE**2, alpha=0.65, label="Raw B–H (scatter)", zorder=5)

print(f"Estimated μ_i (from initial curve) = {mu_i:.1f}")
print(f"Estimated J_s (from initial curve) = {Js:.6f} T")
print(f"c sweep: {C_MIN} → {C_MAX} with {N_C} values")
if EXPORT_ALL:
    print(f"Exported {N_C} CST tables to: {OUT_DIR.resolve()}")

plt.xlabel("H [A/m]", fontsize=FONT_SIZE)
plt.ylabel("B [T]", fontsize=FONT_SIZE)
plt.title("Initial Magnetization Curve — c sweep", fontsize=FONT_SIZE+2)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=LEGEND_SIZE)
plt.xticks(fontsize=FONT_SIZE-2); plt.yticks(fontsize=FONT_SIZE-2)
plt.tight_layout()
plt.show()


Estimated μ_i (from initial curve) = 4505.0
Estimated J_s (from initial curve) = 0.388054 T
c sweep: 2 → 3 with 5 values
Exported 5 CST tables to: C:\Python_Sripts\Core_Calculations\out


In [10]:
# === Cell 2: Export CST table for a manually chosen c ===
# Assumes Cell 1 already defined: mu0, analytical_J(), mu_i, Js, H_grid (or H_raw/H_init + H_MAX_FACTOR), pd, np, Path

# ---- User input ----
C_SELECTED = 2.5                        # <--- try your chosen c here
OUT_DIR = Path("out")                     # same folder you used before (or change)
OUT_FILE_SINGLE = OUT_DIR / f"initial_BH_c_{C_SELECTED:.3f}.txt"
N_EXPORT = 500                            # points in exported curve (override if you like)

# ---- Rebuild H_grid if it's not available (safe fallback) ----
if 'H_grid' not in globals():
    # If raw/init data exist, use their range; else fall back to a generic range
    if 'H_raw' in globals() and 'H_init' in globals() and 'H_MAX_FACTOR' in globals():
        Hmax = float(max(np.max(H_raw), np.max(H_init)) * H_MAX_FACTOR)
        H_grid = np.linspace(0.0, max(Hmax, 1.0), N_EXPORT)
    else:
        H_grid = np.linspace(0.0, 1e4, N_EXPORT)  # generic fallback

# ---- Compute model curve for the chosen c ----
B_model = mu0 * H_grid + analytical_J(H_grid, mu_i, Js, float(C_SELECTED))

# ---- Export (CST format: "H [A/m] \t B [T]" with no header) ----
OUT_DIR.mkdir(parents=True, exist_ok=True)
import pandas as pd
pd.DataFrame({"H": H_grid, "B": B_model}).to_csv(
    OUT_FILE_SINGLE, sep="\t", index=False, header=False, float_format="%.9f"
)

print(f"Saved CST curve for c={C_SELECTED:.3f} → {OUT_FILE_SINGLE.resolve()}")

# ---- Optional quick overlay plot (comment out if not needed) ----
import matplotlib.pyplot as plt
plt.figure(figsize=(9,6))
if 'H_init' in globals() and 'B_init' in globals():
    plt.plot(H_init, B_init, lw=2.2, label="Extracted initial (data)")
plt.plot(H_grid, B_model, lw=2.6, label=f"Analytical (c={C_SELECTED:.2f})")
plt.xlabel("H [A/m]"); plt.ylabel("B [T]"); plt.title("Initial B(H) — selected c")
plt.grid(True, alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()


Saved CST curve for c=2.500 → C:\Python_Sripts\Core_Calculations\out\initial_BH_c_2.500.txt


In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
N87 → CST 'User' export + single-term Generalized Debye (Havriliak–Negami) auto-fit

- Exports CST 'User' file: f[Hz], μ′, μ″
- Auto-tunes a CST-safe single HN term (band/weights/α/β/restarts)
- Saves best params + fit plot + leaderboard
- Prints tolerance estimates for μ′ (→ X_L), μ″ (→ ESR_core), and loss scale factor
- NEW: error-vs-frequency plot uses LOG y-axis of ABSOLUTE % error (semilogy)

Kernel uses e^{-jωt}: μ* = μ′ − j μ″ (so μ″ ≥ 0 for passive media).
"""

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

# ---------- Paths ----------
EXCEL_PATH   = Path("N87_Permeability.xlsx")
USER_OUT     = Path("N87_Permeability_User_For_CST.txt")
BEST_PARAMS  = Path("N87_HN_params_best.csv")
BEST_PNG     = Path("N87_HN_fit_best.png")
ERROR_PNG    = Path("N87_HN_fit_errors.png")
LEADERBOARD  = Path("N87_HN_leaderboard.csv")

# ---------- Auto-tune grids ----------
BAND_SPREADS = [(1/12, 8), (1/8, 8), (1/6, 6), (1/4, 4)]
MU_PP_WEIGHTS = [2.0, 3.0, 4.0, 5.0]
ALPHA_MODES   = [("frozen", 0.999), ("free", None)]
BETA_MODES    = [("frozen", 1.0),   ("free",  None)]
RESTARTS      = [(0.5, 0.7, 0.9), (1.0, 1.0, 1.0), (2.0, 1.5, 1.1)]

# ---------- Plot look ----------
X_SCALE = "log"
Y_SCALE = "linear"

# =====================================================================
# Data I/O
# =====================================================================
def load_excel(path: Path):
    df = pd.read_excel(path)
    req = ["Series1.X", "Series1.Y", "Series2.X", "Series2.Y"]
    miss = [c for c in req if c not in df.columns]
    if miss:
        raise ValueError(f"Missing expected columns: {miss}")
    s1 = (df[["Series1.X", "Series1.Y"]]
          .replace([np.inf, -np.inf], np.nan).dropna())
    s1 = s1[s1["Series1.X"] > 0].sort_values("Series1.X").drop_duplicates("Series1.X", keep="last")
    f1_hz   = s1["Series1.X"].to_numpy(float) * 1e3
    mu1_src = s1["Series1.Y"].to_numpy(float)

    s2 = (df[["Series2.X", "Series2.Y"]]
          .replace([np.inf, -np.inf], np.nan).dropna())
    s2 = s2[s2["Series2.X"] > 0].sort_values("Series2.X")
    f_hz = s2["Series2.X"].to_numpy(float) * 1e3
    mu2  = s2["Series2.Y"].to_numpy(float)
    if len(f1_hz) == 0 or len(f_hz) == 0:
        raise ValueError("No valid μ′/μ″ data after cleaning.")
    return f1_hz, mu1_src, f_hz, mu2

def export_user(f_hz, mu1_on_f, mu2, out_path: Path):
    pd.DataFrame({"f_Hz": f_hz, "mu_prime": mu1_on_f, "mu_doubleprime": mu2}) \
      .to_csv(out_path, sep="\t", index=False, header=False, float_format="%.9f")

# =====================================================================
# Helpers
# =====================================================================
def interp_hold(x_src, y_src, x_new):
    return np.interp(x_new, x_src, y_src, left=y_src[0], right=y_src[-1])

def hn_complex(mu_inf, dmu, tau, alpha, beta, f_hz):
    """HN complex μ* with μs = μ∞ + Δμ, using e^{-jωt} (μ″ ≥ 0)."""
    #mu_s = mu_inf + dmu    #If wanted to calculated from data
    mu_s = 2156  # Set your desired value here
    w = 2*np.pi*f_hz
    tau   = max(tau, 1e-15)
    alpha = np.clip(alpha, 1e-6, 0.999999)  # CST-safe
    beta  = np.clip(beta,  1e-6, 1.0)
    jwta  = (-1j*w*tau)**alpha  # sign so μ″ is positive
    denom = (1.0 + jwta)**beta
    return mu_inf + (mu_s - mu_inf)/denom

def overlap_band(f_hz, f1_max):
    return (f_hz <= f1_max)

def auto_band(f_hz, mu2, ov_mask, low_mult, high_mult):
    f_ov, mu2_ov = f_hz[ov_mask], mu2[ov_mask]
    if len(f_ov) < 16:
        raise ValueError("Overlap too small for fitting.")
    p = int(np.nanargmax(mu2_ov))
    f_peak = float(f_ov[p])
    fmin = max(f_ov[0], f_peak*low_mult)
    fmax = min(f_ov[-1], f_peak*high_mult)
    return (f_hz >= fmin) & (f_hz <= fmax) & ov_mask, f_peak

# =====================================================================
# Fitting (with restarts and modes)
# =====================================================================
def fit_once(f_hz, mu1_on_f, mu2, mask, f_peak,
             mu_pp_weight=3.0,
             alpha_mode=("frozen", 0.999),
             beta_mode=("frozen", 1.0),
             restart=(1.0, 1.0, 1.0)):
    eps = 1e-9
    f_fit   = f_hz[mask]
    mu1_fit = mu1_on_f[mask]
    mu2_fit = mu2[mask]

    # seeds from data
    n = len(mu1_fit); k = max(3, n//7)
    mu_low  = float(np.median(mu1_fit[:k]))
    mu_high = float(np.median(mu1_fit[-k:]))
    mu_inf0 = max(1.0, min(mu_high, mu_low)) * restart[2]
    dmu_from_mu   = max(5.0, abs(mu_low - mu_high))
    dmu_from_mupp = max(5.0, 2.0*float(np.nanmax(mu2_fit)))   # Debye identity
    dmu0 = max(dmu_from_mu, dmu_from_mupp) * restart[1]
    tau0 = (1.0/(2*np.pi*f_peak)) * restart[0]

    # variables θ = [mu_inf, dmu, log10_tau, (alpha?), (beta?)]
    th0 = [mu_inf0, dmu0, np.log10(tau0)]
    lb  = [0.2*mu_high,  0.1*dmu0, np.log10(tau0/10)]
    ub  = [2.0*mu_low,   10.0*dmu0, np.log10(tau0*10)]

    alpha_fixed = alpha_mode[1] if alpha_mode[0]=="frozen" else None
    beta_fixed  = beta_mode[1]  if beta_mode[0]=="frozen"  else None
    if alpha_fixed is None:
        th0.append(0.95); lb.append(0.90); ub.append(0.999)
    if beta_fixed is None:
        th0.append(1.0);  lb.append(0.85); ub.append(1.0)

    th0 = np.array(th0, float); lb = np.array(lb, float); ub = np.array(ub, float)
    mu1_scale = max(1.0, np.median(mu1_fit))

    def unpack(theta):
        i=0
        mu_inf = theta[i]; i+=1
        dmu    = theta[i]; i+=1
        tau    = 10.0**theta[i]; i+=1
        alpha  = alpha_fixed if alpha_fixed is not None else theta[i]; i += (0 if alpha_fixed is not None else 1)
        beta   = beta_fixed  if beta_fixed  is not None else theta[i]
        return mu_inf, dmu, tau, alpha, beta

    def resid(theta):
        mu_inf, dmu, tau, alpha, beta = unpack(theta)
        mu = hn_complex(mu_inf, dmu, tau, alpha, beta, f_fit)
        r_real = (mu.real - mu1_fit) / mu1_scale
        mu_i = np.maximum(mu.imag, eps)
        r_imag = np.log10(mu_i) - np.log10(np.maximum(mu2_fit, eps))
        pen = np.minimum(mu.imag, 0.0) * 0.05
        return np.hstack([r_real, mu_pp_weight*r_imag, pen])

    res = least_squares(resid, th0, bounds=(lb, ub),
                        loss="soft_l1", f_scale=0.3,
                        xtol=1e-12, ftol=1e-12, gtol=1e-12,
                        max_nfev=80000)
    mu_inf, dmu, tau, alpha, beta = unpack(res.x)
    alpha = min(alpha, 0.999999)  # CST-safe
    return (mu_inf, dmu, tau, alpha, beta), res.cost

# =====================================================================
# Scoring (fixed metric so different trial weights are comparable)
# =====================================================================
def evaluate_fit(f_hz, mu1_on_f, mu2, params, eval_mask):
    eps = 1e-9
    mu = hn_complex(*params, f_hz[eval_mask])
    mu1_fit = mu1_on_f[eval_mask]
    mu1_scale = max(1.0, np.median(mu1_fit))
    e_r = (mu.real - mu1_fit)/mu1_scale
    rms_r = np.sqrt(np.mean(e_r**2))
    mu2_fit = mu2[eval_mask]
    e_i = (np.log10(np.maximum(mu.imag, eps))
           - np.log10(np.maximum(mu2_fit, eps)))
    rms_i = np.sqrt(np.mean(e_i**2))
    neg_pen = float(np.sum(np.minimum(mu.imag, 0.0)**2)) * 1e-4
    score = rms_r + 3.0*rms_i + neg_pen
    return score, rms_r, rms_i

# =====================================================================
# Main auto-tune
# =====================================================================
def main():
    f1_hz, mu1_src, f_hz, mu2 = load_excel(EXCEL_PATH)
    mu1_on_f = interp_hold(f1_hz, mu1_src, f_hz)
    export_user(f_hz, mu1_on_f, mu2, USER_OUT)

    ov_mask = overlap_band(f_hz, f1_hz.max())

    rows = []
    best = None  # (score, params, mask, details dict)

    for (lb_mult, ub_mult) in BAND_SPREADS:
        mask_band, f_peak = auto_band(f_hz, mu2, ov_mask, lb_mult, ub_mult)
        eval_mask = mask_band.copy()

        for wpp in MU_PP_WEIGHTS:
            for a_mode in ALPHA_MODES:
                for b_mode in BETA_MODES:
                    for rst in RESTARTS:
                        params, ls_cost = fit_once(
                            f_hz, mu1_on_f, mu2, mask_band, f_peak,
                            mu_pp_weight=wpp,
                            alpha_mode=a_mode,
                            beta_mode=b_mode,
                            restart=rst
                        )
                        score, rms_r, rms_i = evaluate_fit(f_hz, mu1_on_f, mu2,
                                                           params, eval_mask)
                        mu_inf, dmu, tau, alpha, beta = params
                        mu_s = mu_inf + dmu
                        row = {
                            "score": score, "rms_mu_prime": rms_r, "rms_log_mu_dblprime": rms_i,
                            "mu_infinity": mu_inf, "mu_static": mu_s, "tau_s": tau,
                            "alpha": alpha, "beta": beta,
                            "band_low_mult": lb_mult, "band_high_mult": ub_mult,
                            "mu_pp_weight": wpp,
                            "alpha_mode": a_mode[0], "beta_mode": b_mode[0],
                            "restart_tau_mult": rst[0], "restart_dmu_mult": rst[1], "restart_muinf_mult": rst[2],
                            "ls_internal_cost": ls_cost,
                            "f_eval_min_Hz": float(f_hz[eval_mask][0]),
                            "f_eval_max_Hz": float(f_hz[eval_mask][-1]),
                            "f_peak_Hz": float(f_peak)
                        }
                        rows.append(row)
                        if (best is None) or (score < best[0]):
                            best = (score, params, eval_mask, row)

    # Save leaderboard
    lb_df = pd.DataFrame(rows).sort_values("score").reset_index(drop=True)
    lb_df.to_csv(LEADERBOARD, index=False)

    # Best result
    best_score, best_params, best_mask, best_row = best
    mu_inf, dmu, tau, alpha, beta = best_params
    mu_s = mu_inf + dmu

    # Save best params
    pd.DataFrame([{
        "mu_infinity": mu_inf,
        "mu_static_default": mu_s,
        "relaxation_time_s": tau,
        "alpha": alpha,
        "beta": beta,
        "score": best_score
    }]).to_csv(BEST_PARAMS, index=False)

    # Plot best (μ curves)
    mu = hn_complex(mu_inf, dmu, tau, alpha, beta, f_hz)
    plt.figure(figsize=(10,8))
    if X_SCALE == "log": plt.xscale("log")
    plt.yscale(Y_SCALE)
    plt.plot(f_hz, mu1_on_f, label="μ' (target)")
    plt.plot(f_hz, mu2,      label="μ'' (target)")
    plt.plot(f_hz, mu.real,  label="μ' (Generalized Debye fit)")
    mupp = mu.imag
    if Y_SCALE == "log":
        mupp = np.where(mupp > 0, mupp, np.nan)
    else:
        mupp = np.maximum(mupp, 0.0)
    plt.plot(f_hz, mupp,     label="μ'' (Generalized Debye fit)")
    if best_mask is not None and np.any(best_mask):
        plt.axvspan(np.min(f_hz[best_mask]), np.max(f_hz[best_mask]),
                    color='0.9', alpha=0.35, label="Fit band")
    if Y_SCALE == "linear":
        plt.ylim(bottom=0)
    plt.xlabel("Frequency [Hz]"); plt.ylabel("Permeability")
    plt.title("N87: CST 'User' Data & Generalized Debye (HN) Fit — BEST")
    plt.legend(); plt.tight_layout(); plt.savefig(BEST_PNG, dpi=150)

    # ---------------- TOLERANCE ESTIMATES ----------------
    eps = 1e-12
    f_fit   = f_hz[best_mask]
    mu1_meas = mu1_on_f[best_mask]
    mu2_meas = mu2[best_mask]
    mu_fit   = hn_complex(mu_inf, dmu, tau, alpha, beta, f_fit)

    # Reactance / inductance error ≈ μ′ error
    rel_err_mu1 = (mu_fit.real - mu1_meas) / np.maximum(mu1_meas, eps)
    mu1_med_err = float(np.median(np.abs(rel_err_mu1)))
    mu1_max_err = float(np.max(np.abs(rel_err_mu1)))

    # Loss / ESR error ≈ μ″ error (where μ″ is significant)
    mu2_pk = float(np.max(mu2_meas))
    loss_mask = mu2_meas > (0.1 * mu2_pk)
    if not np.any(loss_mask):
        loss_mask = np.ones_like(mu2_meas, dtype=bool)
    rel_err_mu2 = (mu_fit.imag - mu2_meas) / np.maximum(mu2_meas, eps)
    mu2_med_err = float(np.median(np.abs(rel_err_mu2[loss_mask])))
    mu2_max_err = float(np.max(np.abs(rel_err_mu2[loss_mask])))

    # Suggested loss scale factors (measured/model)
    loss_ratio = np.maximum(mu2_meas[loss_mask], eps) / np.maximum(mu_fit.imag[loss_mask], eps)
    scale_med  = float(np.median(loss_ratio))
    scale_p90  = float(np.percentile(loss_ratio, 90))

    # ---------------- ERROR VS FREQUENCY (LOG Y, ABS % ERRORS) ----------------
    ov_mask_full = overlap_band(f_hz, f1_hz.max())
    mu_all       = hn_complex(mu_inf, dmu, tau, alpha, beta, f_hz)

    err_mu1_pct = np.full_like(f_hz, np.nan, float)
    err_mu2_pct = np.full_like(f_hz, np.nan, float)
    err_q_pct   = np.full_like(f_hz, np.nan, float)

    # μ′ error where overlap exists
    err_mu1_pct[ov_mask_full] = 100.0 * (mu_all.real[ov_mask_full] - mu1_on_f[ov_mask_full]) / \
                                np.maximum(mu1_on_f[ov_mask_full], eps)

    # μ″ error only where μ″ is meaningful (>= 10% of peak inside overlap)
    mu2_ov = mu2[ov_mask_full]; pk = np.nanmax(mu2_ov)
    good_loss = ov_mask_full & (mu2 >= 0.1 * pk)
    err_mu2_pct[good_loss] = 100.0 * (mu_all.imag[good_loss] - mu2[good_loss]) / \
                             np.maximum(mu2[good_loss], eps)

    # Q error where loss is meaningful
    q_meas = np.full_like(f_hz, np.nan, float)
    q_fit  = np.full_like(f_hz, np.nan, float)
    q_meas[good_loss] = mu1_on_f[good_loss] / np.maximum(mu2[good_loss], eps)
    q_fit[good_loss]  = mu_all.real[good_loss] / np.maximum(mu_all.imag[good_loss], eps)
    err_q_pct[good_loss] = 100.0 * (q_fit[good_loss] - q_meas[good_loss]) / np.maximum(q_meas[good_loss], eps)

    # semilogy of ABSOLUTE % errors
    epsy = 1e-2  # floor = 0.01% to avoid log(0)
    plt.figure(figsize=(12,6))
    plt.xscale("log")
    plt.semilogy(f_hz, np.where(np.isfinite(err_mu1_pct), np.maximum(np.abs(err_mu1_pct), epsy), np.nan),
                 label="|μ′ error| → |X_L| error [%]")
    plt.semilogy(f_hz, np.where(np.isfinite(err_mu2_pct), np.maximum(np.abs(err_mu2_pct), epsy), np.nan),
                 label="|μ″ error| → |ESR_core| error [%]")
    plt.semilogy(f_hz, np.where(np.isfinite(err_q_pct),   np.maximum(np.abs(err_q_pct),   epsy), np.nan),
                 label="|Q error| [%]", linestyle=":")

    # reference lines
    for y in [1, 5, 10]:
        plt.axhline(y, color="0.85", lw=0.8, zorder=0)
        plt.text(f_hz[0]*1.05, y*1.03, f"{y}%", color="0.5", fontsize=9, va="bottom")

    if best_mask is not None and np.any(best_mask):
        plt.axvspan(np.min(f_hz[best_mask]), np.max(f_hz[best_mask]),
                    color='0.9', alpha=0.35, label="Fit band")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Absolute relative error [%] (log scale)")
    plt.title("HN fit error vs frequency (impedance-related)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(ERROR_PNG, dpi=150)

    # Console summary + sanity check
    print("\nBest HN parameters (single term, CST-safe):")
    print(f"  mu_infinity      = {mu_inf:.6g}")
    print(f"  mu_static (def.) = {mu_s:.6g}   (Δμ = {dmu:.6g})")
    print(f"  tau [s]          = {tau:.6e}")
    print(f"  alpha            = {alpha:.6g}  (< 1.0)")
    print(f"  beta             = {beta:.6g}")
    print(f"  Score            = {best_score:.6g}")
    f_peak_pred = 1.0/(2*np.pi*tau)
    print(f"  Sanity: predicted f_peak ≈ {f_peak_pred:.3g} Hz,  μ\"_max ≈ {0.5*dmu:.2f}")

    print("\nEstimated tolerances in the fit band:")
    print(f"  Reactance / inductance (≈ μ′): median ±{mu1_med_err*100:.1f}%, max ±{mu1_max_err*100:.1f}%")
    print(f"  Core loss / ESR (≈ μ″):       median ±{mu2_med_err*100:.1f}%, max ±{mu2_max_err*100:.1f}%")
    print(f"  Suggested loss scale (meas/model): median ×{scale_med:.2f}, 90th pct ×{scale_p90:.2f}")

    print("\nFiles:")
    print(f"  User export      : {USER_OUT}")
    print(f"  Best params CSV  : {BEST_PARAMS}")
    print(f"  Leaderboard CSV  : {LEADERBOARD}")
    print(f"  Best-fit plot    : {BEST_PNG}")
    print(f"  Error plot       : {ERROR_PNG}")
    print("\nPaste μ∞, μs (default), τ, α, β into CST → Magnetic → Dispersion → Generalized Debye,")
    print("then enable Field dependency → B–H curve to include saturation.")
    print("Use the loss scale factor to bracket temperature rise/efficiency if needed.")

if __name__ == "__main__":
    main()



Best HN parameters (single term, CST-safe):
  mu_infinity      = 42.2
  mu_static (def.) = 11371   (Δμ = 11328.8)
  tau [s]          = 5.164670e-08
  alpha            = 0.999  (< 1.0)
  beta             = 1
  Score            = 0.959507
  Sanity: predicted f_peak ≈ 3.08e+06 Hz,  μ"_max ≈ 5664.41

Estimated tolerances in the fit band:
  Reactance / inductance (≈ μ′): median ±22.9%, max ±773.2%
  Core loss / ESR (≈ μ″):       median ±38.4%, max ±103.3%
  Suggested loss scale (meas/model): median ×1.49, 90th pct ×1.80

Files:
  User export      : N87_Permeability_User_For_CST.txt
  Best params CSV  : N87_HN_params_best.csv
  Leaderboard CSV  : N87_HN_leaderboard.csv
  Best-fit plot    : N87_HN_fit_best.png
  Error plot       : N87_HN_fit_errors.png

Paste μ∞, μs (default), τ, α, β into CST → Magnetic → Dispersion → Generalized Debye,
then enable Field dependency → B–H curve to include saturation.
Use the loss scale factor to bracket temperature rise/efficiency if needed.
